In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg
from scipy.interpolate import interp1d
from jupyterthemes import jtplot
import seaborn as sns

jtplot.style(theme = 'gruvboxd')
%matplotlib inline

In [ ]:
He_lin = np.arange(0,max_depth[0] + 1,1)

In [2]:
def matrix_vals(He_lin):
    dt_hours = 12
    siy = 86400 * 365.25
    thermnz = len(He_lin)
    thermdz = He_lin[-1] / (thermnz - 1)
    thermdt = 86400 / (24 / dt_hours) 
    thermnt = int(365 * 24 / dt_hours)
    alpha = 1.5e-6
    kappa = alpha * 365 * 24 * 60 * 60                                     # annual diffusivity (for analytical T solution) 
    
    return dt_hours, siy, thermnz, thermdz, thermdt, thermnt, alpha, kappa 

In [ ]:
dt_hours, siy, thermnz, thermdz, thermdt, thermnt, alpha, kappa = matrix_vals(He_lin)

In [6]:
def lhs_operator(N, sigma):
                                                        
    D = np.diag(2.0 * (1.0 + 1.0 / sigma) * np.ones(N))        # Operator diagonal
    D[-1, -1] = 1.0 + 2.0 / sigma                               # Neumann condition for the last element.
    U = np.diag(-1.0 * np.ones(N - 1), k=1)                    # Upper diagonal
    L = np.diag(-1.0 * np.ones(N - 1), k=-1)                   # Lower diagonal

    A = D + U + L                                                 # Assemble the operator.

    return A

def Temp_fun(MAT, AMP, z, tiy):
    
    kappa = 1.5*10**-2 * 365 * 24 * 60 * 60
    Py = 1
    t = tiy / thermnt
    Amplitude = AMP
    AMP = AMP
    T = MAT - Amplitude * np.exp(-z * np.sqrt(np.pi / (kappa * Py))) * np.cos(2 * np.pi * t / Py - z * np.sqrt(np.pi / (kappa * Py)))    

    return T
    
def rhs_vector(T, MAT, AMP, sigma, qdx, n):
    
    b = T[:-2] + 2.0 * (1.0 / sigma - 1.0) * T[1:-1] + T[2:]                    # Set Dirichlet condition
    b[0] += Temp_fun(MAT, AMP, 0, n)                                              # Set Neumann condition
    b[-1] += qdx
    
    return b
    
def crank_nicolson(T0, MAT, AMP, nt, dt, dx, alpha, q):
            
    # Create the implicit operator of the system
    sigma = alpha * dt / dx**2
    A = lhs_operator(len(T0) - 2, sigma)                                    # Integrate through time
    T = T0.copy()                                                           
    T_mat = np.empty((thermnz,365))

    for n in range(nt):
        b = rhs_vector(T, MAT, AMP, sigma, q * dx, n)                                    # Generate RHS
        T[1:-1] = linalg.solve(A, b)                                        # Solve the system
        T[-1] = T[-2] + q * dx                                              # Apply Neumann BC
        
#         if n <= (74*2) or n >= (288 * 2):
#             T[0] = 0
#         else:
#             T[0] = Temp_fun(MAT, AMP, 0, n)
        
        T[0] = Temp_fun(MAT, AMP, 0, n)

        if n % (24 / dt_hours) == 0:
            T_mat[:,int(n/(24 / dt_hours))] = T    
        
        # T_mat[:,n] = T
        
    return T, T_mat

def diff_calc(T):
    
    Diff = [10**(-3928.1*(1/(T[i])) - 2.9315) for i in range(len(T))]              # cm^^2 * s^-1
    
    return Diff
    
def EDT_calculation(MAT, AMP, T0):    
    Ea = 112 * 1e3
    R = 8.3144621

    # alpha_m = 1.5*10**-6                                                               
    alpha_cm = alpha / 10**-4    

    q = 0.0  
    
    T0 = T0
    
    Temps_output = crank_nicolson(T0, MAT, AMP, thermnt, thermdt, thermdz, alpha_cm, q)[1]
    Temps = Temps_output
    
    depth_mean_Ts = [None] * np.shape(Temps)[0]
    depth_integ = [None] * np.shape(Temps)[0]
    
    exper = lambda t: np.exp(-Ea / (R * (t + 273.15)))
    exps = np.array([exper(Tempsi) for Tempsi in Temps])
    EDTs = [(-Ea / R / np.log(np.mean(exps[i,:]))) for i in range(len(He_lin))]
    
    
    depth_mean_Ts = [(np.mean(Temps[i,:])) for i in range(len(Temps))]
    plus = lambda t: t + 273.15
    depth_mean_Ts = [plus(depth_mean_Tsi) for depth_mean_Tsi in depth_mean_Ts]
    
    Diff_z = diff_calc(EDTs)
    
    return Temps_output, Temps, depth_mean_Ts, EDTs, Diff_z, He_lin